In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# It returns all list of individual words of the document text.
def get_individual_words(document_text):
    full_text=document_text
    full_text=full_text.strip()
    full_text=full_text.replace("\n", ",")
    full_text=full_text.replace("\t", ",")
    full_text=full_text.replace(":", ",")
    full_text=full_text.replace(";", ",")
    full_text=full_text.replace(" ", ",")
    full_text=full_text.replace(" ", ",")
    full_text=full_text.replace("-", ",")
    full_text=full_text.replace("*", ",")
    full_text=full_text.replace("&", ",")
    full_text=full_text.replace("+", ",")
    full_text=full_text.replace("$", ",")
    full_text=full_text.replace("/", ",")
    full_text=full_text.replace("%", ",")

    full_text=full_text.lower()
    full_words=full_text.split(',')
    
    return full_words;

# it returns the documents and it's labels from corpus 
def get_documents_labels(file_path):
    
    file1 = open(file_path, 'r')
    Lines = file1.readlines()
    
    documents=[]
    labels=[]
    for line in Lines:
        temp=line.split('\t')
        #print(temp)
        documents.append(temp[0])
        if(temp[1]=='0\n'):
            labels.append(-1)
        elif(temp[1]=='1\n'):
            labels.append(1)
    labels=np.array(labels)
    return documents,labels

# It returns list of all the words of the documents with repeatation.
def get_all_words_of_documents(documents):
    full_text=''
    
    for i in documents:
        full_text=full_text+" "+i
        
    full_words=get_individual_words(full_text)
    
    for i in range(len(full_words)):
        full_words[i]=full_words[i].lower()
    
    
    full_words_updated=[]
    for individual_word in full_words:
        if(len(individual_word)!=0 and individual_word!=' '):
            full_words_updated.append(individual_word)
    return full_words_updated;


In [3]:
# It returns the term frequnecy of the term_t in the document_text
def tf(term_t,document_text):
    term_t=term_t.lower()
    full_words=get_individual_words(document_text)
    #print(full_words)
    term_t_count=0
    if(term_t in full_words):
        for i in full_words:
           if(i==term_t):
               term_t_count=term_t_count+1
    #print(term_t_count,len(full_words))
    return term_t_count/len(full_words)



import math

# it returns the tf-idf of the term_t of the document_text.
def tf_idf_single_term(term_t,document_text):
    term_t=term_t.lower()
    index=term_dictionary[term_t]
    term_freq=tf(term_t,document_text)
    document_freq=Document_Frequency[index]
    #print(term_freq,document_freq)
    if(document_freq>0):
        return term_freq*math.log(total_words_N/document_freq)
    else:
        return term_freq*math.log(total_words_N/1+document_freq)
    

# it returns the tf-idf of the document_text.
# it will be used to represent the document as a feature vector.
def tf_idf(document_text):
    feature_vector=np.zeros(total_unique_words_V,)
    for i in range(len(dictionary)):
        feature_vector[i]=tf_idf_single_term(dictionary[i],document_text)
    return feature_vector


In [4]:
def higher_dimension_PCA(X):
    # Data Centering
    X_mean=np.mean(X,axis=0)
    X_centered=X-X_mean
    
    #checking the mean of the centered Data.
    #print(np.mean(X_centered,axis=0))
    #print(np.sum(np.mean(X_centered,axis=0)))

    # we are calculating the eigen values of the X@XT for calculating the eigen values of the XT@X
    
    N=X.shape[0]
    temp_outer_product=(1/N)*(X_centered@X_centered.T)
    temp_eigen_values,temp_eigen_vectors=np.linalg.eigh(temp_outer_product)
    
    idx = np.argsort(-temp_eigen_values)
    temp_eigen_values = temp_eigen_values[idx]
    temp_eigen_vectors = temp_eigen_vectors[:,idx]
        
    
    #print(temp_eigen_values.shape)
    #print(temp_eigen_vectors.shape)
    
    
    # For calculating the eigenvalues of the XT@X
    
    #print(temp_eigen_values)
    print(N,temp_eigen_values)
    norms_of_eigen_vectors=np.sqrt(N*temp_eigen_values)
    eigen_vectors=X_centered.T@temp_eigen_vectors
    
    for i in range(eigen_vectors.shape[1]):
        eigen_vectors[:,i]=eigen_vectors[:,i]/norms_of_eigen_vectors[i]
    
    print('Eigen vectors shape',eigen_vectors.shape)

    return eigen_vectors


In [5]:
def data_centering(X):
    X_mean=np.mean(X,axis=0)
    X_centered=X-X_mean
    return X_centered 

In [6]:
file_path='movie_reviews/movieReviews1000.txt'

# reading a documents and labels from the corpus (text file) 
documents,labels=get_documents_labels(file_path)

# full words of all the documents with repeatation. 
full_words=get_all_words_of_documents(documents)

dictionary = list(set(full_words))
dictionary.sort()


In [7]:
total_unique_words_V=len(dictionary)
total_words_N=len(full_words)
total_documents_D=len(documents)

print('total no of documents:',total_documents_D)
print('total no of words in the documents(N):',total_words_N)
print('total no of unique words(V):',total_unique_words_V)

total no of documents: 1000
total no of words in the documents(N): 14436
total no of unique words(V): 3139


In [8]:
#making a term dictionary

term_dictionary={}
for i in range(len(dictionary)):
    term_dictionary[dictionary[i]]=i

In [9]:
#print term-frequency of the slow word in the following sentence.

print(documents[0])
print("term-frequency of word slow:",tf("slow",documents[0]))

A very very very slow-moving aimless movie about a distressed drifting young man  
term-frequency of word slow: 0.07142857142857142


In [10]:
Term_Frequecy=np.zeros((total_unique_words_V,total_documents_D),dtype='float64')

for i in range(len(dictionary)):
    for j in range(len(documents)):
        Term_Frequecy[i][j]=tf(dictionary[i],documents[j])

print("Term_Frequecy calculation was completed---------------------")

Term_Frequecy calculation was completed---------------------


In [11]:
#document frequecny calculations
# execution is only once for code.............. 

       
Document_Frequency=np.zeros(total_unique_words_V)

for i in range(len(dictionary)):
    temp=Term_Frequecy[i,:]
    document_indices=np.where(temp>0)
    document_indices=np.array(document_indices)
    Document_Frequency[i]=document_indices.shape[1]

print("Document Frequency calculation was completed---------------------")

Document Frequency calculation was completed---------------------


In [12]:
document_id=np.random.randint(1,1000)
document_vector=tf_idf(documents[document_id])

print('Document text:\n',documents[document_id])
print('tf-idf array:\n',document_vector )

print("Following elements of document vectors are greater than zero:\n")
indices=np.where(document_vector>0)
print("index-----tf-idf")
for i  in indices[0]:
    print(i,"-----",document_vector[i])

Document text:
 The movie seemed a little slow at first  
tf-idf array:
 [0. 0. 0. ... 0. 0. 0.]
Following elements of document vectors are greater than zero:

index-----tf-idf
44 ----- 0.4689350322977786
207 ----- 0.7008985565263197
1063 ----- 0.8974481366205386
1632 ----- 0.8430333777133079
1804 ----- 0.5544771011575024
2372 ----- 1.0598585096368212
2474 ----- 0.973215112066828
2712 ----- 0.41763884694487186


In [13]:
# we are representing the document as a vector
# it representing the corpus of feature vectors.
    
document_vector=[]

for i in documents:
    document_vector.append(tf_idf(i))

document_vector=np.array(document_vector)
document_vector.shape

(1000, 3139)

In [14]:
from sklearn.decomposition import PCA

K=10
document_vector_centered=data_centering(document_vector)
print('Data_shape',document_vector_centered.shape)
pca = PCA(n_components = K)
projected_data = pca.fit_transform(document_vector_centered)

print('projected_data>shape',projected_data.shape)

Data_shape (1000, 3139)
projected_data>shape (1000, 10)


In [15]:
def random_sampling(X,X_labels,total_no_samples):
    list_indices=list(range(X.shape[0]))
    np.random.shuffle(list_indices)	
    train_indices=list_indices[0:total_no_samples]
    test_indices=list_indices[total_no_samples:]
    
    train_data=(X[train_indices],X_labels[train_indices])
    test_data=(X[test_indices],X_labels[test_indices])
    return train_data,test_data

In [16]:
def sequential_sampling(X,X_labels,total_no_samples):
    train_data=(X[0:total_no_samples],X_labels[0:total_no_samples])
    test_data=(X[total_no_samples:],X_labels[total_no_samples:])
    return train_data,test_data

In [17]:
from libsvm.svm import *
from libsvm.svmutil import *

In [18]:
def svm_cusomized(train_data,labels,C,epsilon,kernel_type):
    prob = svm_problem(labels,train_data)

    param = svm_parameter()
    param.C = C
    param.nu=epsilon
    
    if(kernel_type=='LINEAR'):
        param.kernel_type = LINEAR
    elif(kernel_type=='POLY'):
        param.kernel_type = POLY
    elif(kernel_type=='RBF'):
        param.kernel_type = RBF
    elif(kernel_type=='SIGMOID'):
        param.kernel_type = SIGMOID
    
    m = svm_train(prob, param)

    return m

In the following we are doing lot of experiments with various $kernels$ and with different $C$ values.

In the training data we have totally $1000$ samples.

We are using first $750$ samples for traing.

We are using remaing $250$ samples for testing.

In [68]:
#taking the first 750 samples for training the data.
no_training_samples=750
(train_X,train_labels),(test_X,test_labels)=sequential_sampling(projected_data,labels,no_training_samples)

In [170]:
statastics_linear=[]

In [189]:
C=1000000
epsilon=0.5
kernel_type='LINEAR'

m=svm_cusomized(train_X,train_labels,C,epsilon,kernel_type)

print("For training:")
traing_accuracy=svm_predict(train_labels,train_X,m)[1][0]
print("For testing:")
testing_accuracy=svm_predict(test_labels,test_X,m)[1][0]

print(traing_accuracy,testing_accuracy)

statastics_linear.append([kernel_type,C,epsilon,traing_accuracy,testing_accuracy,m.get_nr_sv()])


For training:
Accuracy = 57.6% (432/750) (classification)
For testing:
Accuracy = 64% (160/250) (classification)
57.599999999999994 64.0


In [185]:
for i in statastics_linear:
    print(i)

['LINEAR', 1, 0.5, 57.199999999999996, 48.0, 666]
['LINEAR', 10, 0.5, 58.13333333333334, 52.800000000000004, 647]
['LINEAR', 100, 0.5, 60.13333333333334, 54.0, 639]
['LINEAR', 1000, 0.5, 60.266666666666666, 54.400000000000006, 634]
['LINEAR', 10000, 0.5, 60.266666666666666, 54.400000000000006, 634]
['LINEAR', 100000, 0.5, 57.333333333333336, 41.199999999999996, 600]
['LINEAR', 1000000, 0.5, 57.599999999999994, 64.0, 609]
['LINEAR', 0.01, 0.5, 53.733333333333334, 38.800000000000004, 696]
['LINEAR', 0.001, 0.5, 53.733333333333334, 38.800000000000004, 695]
['LINEAR', 0.0001, 0.5, 53.733333333333334, 38.800000000000004, 694]
['LINEAR', 1e-05, 0.5, 53.733333333333334, 38.800000000000004, 694]
['LINEAR', 1e-06, 0.5, 53.733333333333334, 38.800000000000004, 694]
['LINEAR', 1e-07, 0.5, 53.733333333333334, 38.800000000000004, 694]
['LINEAR', 1e-08, 0.5, 53.733333333333334, 38.800000000000004, 694]


In [80]:
statastics_poly=[]

In [188]:
C=100000000
epsilon=0.5
kernel_type='POLY'

m=svm_cusomized(train_X,train_labels,C,epsilon,kernel_type)

print("For training:")
traing_accuracy=svm_predict(train_labels,train_X,m)[1][0]
print("For testing:")
testing_accuracy=svm_predict(test_labels,test_X,m)[1][0]

print(traing_accuracy,testing_accuracy)

statastics_poly.append([kernel_type,C,epsilon,traing_accuracy,testing_accuracy,m.get_nr_sv()])

For training:
Accuracy = 73.4667% (551/750) (classification)
For testing:
Accuracy = 55.6% (139/250) (classification)
73.46666666666667 55.60000000000001


In [100]:
for i in statastics_poly:
    print(i)

['POLY', 1, 0.5, 54.0, 39.2, 694]
['POLY', 10, 0.5, 54.666666666666664, 39.2, 691]
['POLY', 100, 0.5, 55.60000000000001, 40.0, 686]
['POLY', 1000, 0.5, 56.8, 40.400000000000006, 672]
['POLY', 10000, 0.5, 58.666666666666664, 41.199999999999996, 666]
['POLY', 100000, 0.5, 60.13333333333334, 42.8, 646]
['POLY', 1000000, 0.5, 65.86666666666666, 44.4, 636]
['POLY', 10000000, 0.5, 72.8, 54.400000000000006, 595]
['POLY', 100000000, 0.5, 73.46666666666667, 55.60000000000001, 569]
['POLY', 0.1, 0.5, 54.0, 39.2, 696]
['POLY', 0.01, 0.5, 53.86666666666666, 38.800000000000004, 697]
['POLY', 0.001, 0.5, 53.733333333333334, 38.800000000000004, 694]
['POLY', 0.0001, 0.5, 53.733333333333334, 38.800000000000004, 694]
['POLY', 1e-05, 0.5, 53.733333333333334, 38.800000000000004, 694]
['POLY', 1e-06, 0.5, 53.733333333333334, 38.800000000000004, 694]
['POLY', 1e-07, 0.5, 53.733333333333334, 38.800000000000004, 694]


In [120]:
statastics_rbf=[]

In [187]:
C=100000
epsilon=0.5
kernel_type='RBF'

m=svm_cusomized(train_X,train_labels,C,epsilon,kernel_type)

print("For training:")
traing_accuracy=svm_predict(train_labels,train_X,m)[1][0]
print("For testing:")
testing_accuracy=svm_predict(test_labels,test_X,m)[1][0]

print(traing_accuracy,testing_accuracy)

statastics_rbf.append([kernel_type,C,epsilon,traing_accuracy,testing_accuracy,m.get_nr_sv()])

For training:
Accuracy = 69.0667% (518/750) (classification)
For testing:
Accuracy = 62.4% (156/250) (classification)
69.06666666666666 62.4


In [137]:
for i in statastics_rbf:
    print(i)

['RBF', 1, 0.5, 56.53333333333334, 41.6, 691]
['RBF', 10, 0.5, 58.266666666666666, 49.2, 662]
['RBF', 100, 0.5, 61.73333333333333, 60.4, 648]
['RBF', 1000, 0.5, 65.06666666666666, 62.8, 634]
['RBF', 10000, 0.5, 64.66666666666666, 62.0, 612]
['RBF', 100000, 0.5, 69.06666666666666, 62.4, 597]
['RBF', 1000000, 0.5, 74.66666666666667, 60.4, 559]
['RBF', 0.1, 0.5, 53.733333333333334, 38.800000000000004, 699]
['RBF', 0.01, 0.5, 53.733333333333334, 38.800000000000004, 698]
['RBF', 0.001, 0.5, 53.733333333333334, 38.800000000000004, 695]
['RBF', 0.0001, 0.5, 53.733333333333334, 38.800000000000004, 694]
['RBF', 1e-05, 0.5, 53.733333333333334, 38.800000000000004, 694]
['RBF', 1e-06, 0.5, 53.733333333333334, 38.800000000000004, 694]
['RBF', 1e-07, 0.5, 53.733333333333334, 38.800000000000004, 694]
['RBF', 10000000, 0.5, 66.26666666666667, 57.199999999999996, 376]


In [143]:
statastics_sigmoid=[]

In [186]:
C=1000000
epsilon=0.5
kernel_type='SIGMOID'

m=svm_cusomized(train_X,train_labels,C,epsilon,kernel_type)

print("For training:")
traing_accuracy=svm_predict(train_labels,train_X,m)[1][0]
print("For testing:")
testing_accuracy=svm_predict(test_labels,test_X,m)[1][0]

print(traing_accuracy,testing_accuracy)

statastics_sigmoid.append([kernel_type,C,epsilon,traing_accuracy,testing_accuracy,m.get_nr_sv()])

For training:
Accuracy = 59.3333% (445/750) (classification)
For testing:
Accuracy = 55.2% (138/250) (classification)
59.333333333333336 55.2


In [161]:
for i in statastics_sigmoid:
    print(i)

['SIGMOID', 1, 0.5, 56.00000000000001, 40.0, 692]
['SIGMOID', 10, 0.5, 57.99999999999999, 48.4, 660]
['SIGMOID', 100, 0.5, 59.73333333333334, 54.400000000000006, 527]
['SIGMOID', 1000, 0.5, 53.86666666666666, 54.0, 369]
['SIGMOID', 10000, 0.5, 57.199999999999996, 50.4, 336]
['SIGMOID', 100000, 0.5, 57.333333333333336, 52.800000000000004, 328]
['SIGMOID', 1000000, 0.5, 59.333333333333336, 55.2, 312]
['SIGMOID', 10000000, 0.5, 59.333333333333336, 55.2, 312]
['SIGMOID', 100000000, 0.5, 59.333333333333336, 55.2, 312]
['SIGMOID', 0.1, 0.5, 53.733333333333334, 38.800000000000004, 696]
['SIGMOID', 0.01, 0.5, 53.733333333333334, 38.800000000000004, 696]
['SIGMOID', 0.001, 0.5, 53.733333333333334, 38.800000000000004, 694]
['SIGMOID', 0.0001, 0.5, 53.733333333333334, 38.800000000000004, 694]
['SIGMOID', 1e-05, 0.5, 53.733333333333334, 38.800000000000004, 694]
['SIGMOID', 1e-06, 0.5, 53.733333333333334, 38.800000000000004, 694]
['SIGMOID', 1e-07, 0.5, 53.733333333333334, 38.800000000000004, 694]


The following are the results of above experiment.
In the above experiments, the training data is first 750 samples and remaining 250 samples are testing data.


<table>
    
<tr><td>Kernel</td><td> C</td><td> Epsilon</td><td> Training Accuracy</td><td> Testing Accuracy</td><td> No of support vectors</td></tr>

<tr><td>'LINEAR'</td><td> 1</td><td> 0.5</td><td> 57.199999999999996</td><td> 48.0</td><td> 666</td></tr>
<tr><td>'LINEAR'</td><td> 10</td><td> 0.5</td><td> 58.13333333333334</td><td> 52.800000000000004</td><td> 647</td></tr>
<tr><td>'LINEAR'</td><td> 100</td><td> 0.5</td><td> 60.13333333333334</td><td> 54.0</td><td> 639</td></tr>
<tr><td>'LINEAR'</td><td> 1000</td><td> 0.5</td><td> 60.266666666666666</td><td> 54.400000000000006</td><td> 634</td></tr>
<tr><td>'LINEAR'</td><td> 10000</td><td> 0.5</td><td> 60.266666666666666</td><td> 54.400000000000006</td><td> 634</td></tr>
<tr><td>'LINEAR'</td><td> 100000</td><td> 0.5</td><td> 57.333333333333336</td><td> 41.199999999999996</td><td> 600</td></tr>
<tr><td>'LINEAR'</td><td> 1000000</td><td> 0.5</td><td> 57.599999999999994</td><td> 64.0</td><td> 609</td></tr>
<tr><td>'LINEAR'</td><td> 0.01</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 696</td></tr>
<tr><td>'LINEAR'</td><td> 0.001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 695</td></tr>
<tr><td>'LINEAR'</td><td> 0.0001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'LINEAR'</td><td> 1e-05</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'LINEAR'</td><td> 1e-06</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'LINEAR'</td><td> 1e-07</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'LINEAR'</td><td> 1e-08</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'POLY'</td><td> 1</td><td> 0.5</td><td> 54.0</td><td> 39.2</td><td> 694</td></tr>
<tr><td>'POLY'</td><td> 10</td><td> 0.5</td><td> 54.666666666666664</td><td> 39.2</td><td> 691</td></tr>
<tr><td>'POLY'</td><td> 100</td><td> 0.5</td><td> 55.60000000000001</td><td> 40.0</td><td> 686</td></tr>
<tr><td>'POLY'</td><td> 1000</td><td> 0.5</td><td> 56.8</td><td> 40.400000000000006</td><td> 672</td></tr>
<tr><td>'POLY'</td><td> 10000</td><td> 0.5</td><td> 58.666666666666664</td><td> 41.199999999999996</td><td> 666</td></tr>
<tr><td>'POLY'</td><td> 100000</td><td> 0.5</td><td> 60.13333333333334</td><td> 42.8</td><td> 646</td></tr>
<tr><td>'POLY'</td><td> 1000000</td><td> 0.5</td><td> 65.86666666666666</td><td> 44.4</td><td> 636</td></tr>
<tr><td>'POLY'</td><td> 10000000</td><td> 0.5</td><td> 72.8</td><td> 54.400000000000006</td><td> 595</td></tr>
<tr><td>'POLY'</td><td> 100000000</td><td> 0.5</td><td> 73.46666666666667</td><td> 55.60000000000001</td><td> 569</td></tr>
<tr><td>'POLY'</td><td> 0.1</td><td> 0.5</td><td> 54.0</td><td> 39.2</td><td> 696</td></tr>
<tr><td>'POLY'</td><td> 0.01</td><td> 0.5</td><td> 53.86666666666666</td><td> 38.800000000000004</td><td> 697</td></tr>
<tr><td>'POLY'</td><td> 0.001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'POLY'</td><td> 0.0001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'POLY'</td><td> 1e-05</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'POLY'</td><td> 1e-06</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'POLY'</td><td> 1e-07</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'RBF'</td><td> 1</td><td> 0.5</td><td> 56.53333333333334</td><td> 41.6</td><td> 691</td></tr>
<tr><td>'RBF'</td><td> 10</td><td> 0.5</td><td> 58.266666666666666</td><td> 49.2</td><td> 662</td></tr>
<tr><td>'RBF'</td><td> 100</td><td> 0.5</td><td> 61.73333333333333</td><td> 60.4</td><td> 648</td></tr>
<tr><td>'RBF'</td><td> 1000</td><td> 0.5</td><td> 65.06666666666666</td><td> 62.8</td><td> 634</td></tr>
<tr><td>'RBF'</td><td> 10000</td><td> 0.5</td><td> 64.66666666666666</td><td> 62.0</td><td> 612</td></tr>
<tr><td>'RBF'</td><td> 100000</td><td> 0.5</td><td> 69.06666666666666</td><td> 62.4</td><td> 597</td></tr>
<tr><td>'RBF'</td><td> 1000000</td><td> 0.5</td><td> 74.66666666666667</td><td> 60.4</td><td> 559</td></tr>
<tr><td>'RBF'</td><td> 0.1</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 699</td></tr>
<tr><td>'RBF'</td><td> 0.01</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 698</td></tr>
<tr><td>'RBF'</td><td> 0.001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 695</td></tr>
<tr><td>'RBF'</td><td> 0.0001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'RBF'</td><td> 1e-05</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'RBF'</td><td> 1e-06</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'RBF'</td><td> 1e-07</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'RBF'</td><td> 10000000</td><td> 0.5</td><td> 66.26666666666667</td><td> 57.199999999999996</td><td> 376</td></tr>
<tr><td>'SIGMOID'</td><td> 1</td><td> 0.5</td><td> 56.00000000000001</td><td> 40.0</td><td> 692</td></tr>
<tr><td>'SIGMOID'</td><td> 10</td><td> 0.5</td><td> 57.99999999999999</td><td> 48.4</td><td> 660</td></tr>
<tr><td>'SIGMOID'</td><td> 100</td><td> 0.5</td><td> 59.73333333333334</td><td> 54.400000000000006</td><td> 527</td></tr>
<tr><td>'SIGMOID'</td><td> 1000</td><td> 0.5</td><td> 53.86666666666666</td><td> 54.0</td><td> 369</td></tr>
<tr><td>'SIGMOID'</td><td> 10000</td><td> 0.5</td><td> 57.199999999999996</td><td> 50.4</td><td> 336</td></tr>
<tr><td>'SIGMOID'</td><td> 100000</td><td> 0.5</td><td> 57.333333333333336</td><td> 52.800000000000004</td><td> 328</td></tr>
<tr><td>'SIGMOID'</td><td> 1000000</td><td> 0.5</td><td> 59.333333333333336</td><td> 55.2</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 10000000</td><td> 0.5</td><td> 59.333333333333336</td><td> 55.2</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 100000000</td><td> 0.5</td><td> 59.333333333333336</td><td> 55.2</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 0.1</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 696</td></tr>
<tr><td>'SIGMOID'</td><td> 0.01</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 696</td></tr>
<tr><td>'SIGMOID'</td><td> 0.001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'SIGMOID'</td><td> 0.0001</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'SIGMOID'</td><td> 1e-05</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'SIGMOID'</td><td> 1e-06</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
<tr><td>'SIGMOID'</td><td> 1e-07</td><td> 0.5</td><td> 53.733333333333334</td><td> 38.800000000000004</td><td> 694</td></tr>
​</table>


The following results are the best of the above experiments.


<table>
<tr><td>Kernel</td><td>C</td><td>Epsilon</td><td>Traing Accuracy</td><td>Testing Accuracy</td><td>NoSupport Vectors</td></tr>
<tr><td>'LINEAR'</td><td> 1000</td><td> 0.5</td><td> 60.266666666666666</td><td> 54.400000000000006</td><td> 634</td></tr>
<tr><td>'LINEAR'</td><td> 1000000</td><td> 0.5</td><td> 57.599999999999994</td><td> 64.0</td><td> 609</td></tr>
<tr><td>'POLY'</td><td> 100000000</td><td> 0.5</td><td> 73.46666666666667</td><td> 55.60000000000001</td><td> 569</td></tr>
<tr><td>'RBF'</td><td> 100000</td><td> 0.5</td><td> 69.06666666666666</td><td> 62.4</td><td> 597</td></tr>
<tr><td>'SIGMOID'</td><td> 1000000</td><td> 0.5</td><td> 59.333333333333336</td><td> 55.2</td><td> 312</td></tr>
</table>


We have done the experiment with random sampling also.

It means, we have selected $750$ samples randomly and we use these 750 samples as training set. 

Remaining $250$ samples are used for the testing.

The following are the results of those experiments.

In [34]:
def svm_experiment(train_X,train_labels,kernel_type):
    statastics=[]
    C=0.000001
    for i in range(14):
        epsilon=0.5
        m=svm_cusomized(train_X,train_labels,C,epsilon,kernel_type)

        print("For training:")
        traing_accuracy=svm_predict(train_labels,train_X,m)[1][0]
        print("For testing:")
        testing_accuracy=svm_predict(test_labels,test_X,m)[1][0]


        print([kernel_type,C,epsilon,traing_accuracy,testing_accuracy,m.get_nr_sv()])

        statastics.append([kernel_type,C,epsilon,traing_accuracy,testing_accuracy,m.get_nr_sv()])
        C=C*10

    for i in statastics:
        print(i)

In [38]:
no_training_samples=750
(train_X,train_labels),(test_X,test_labels)=random_sampling(projected_data,labels,no_training_samples)

kernel_type='LINEAR'
svm_experiment(train_X,train_labels,kernel_type)

For training:
Accuracy = 51.6% (387/750) (classification)
For testing:
Accuracy = 45.2% (113/250) (classification)
['LINEAR', 1e-06, 0.5, 51.6, 45.2, 726]
For training:
Accuracy = 51.6% (387/750) (classification)
For testing:
Accuracy = 45.2% (113/250) (classification)
['LINEAR', 9.999999999999999e-06, 0.5, 51.6, 45.2, 726]
For training:
Accuracy = 51.6% (387/750) (classification)
For testing:
Accuracy = 45.2% (113/250) (classification)
['LINEAR', 9.999999999999999e-05, 0.5, 51.6, 45.2, 727]
For training:
Accuracy = 51.6% (387/750) (classification)
For testing:
Accuracy = 45.2% (113/250) (classification)
['LINEAR', 0.001, 0.5, 51.6, 45.2, 727]
For training:
Accuracy = 51.8667% (389/750) (classification)
For testing:
Accuracy = 45.2% (113/250) (classification)
['LINEAR', 0.01, 0.5, 51.866666666666674, 45.2, 727]
For training:
Accuracy = 58.4% (438/750) (classification)
For testing:
Accuracy = 54% (135/250) (classification)
['LINEAR', 0.1, 0.5, 58.4, 54.0, 705]
For training:
Accuracy = 6

The following is are results of the above experiments (which are trained on random sampled data).

<table>

<tr><td>Kernel</td><td>C</td><td>Epsilon</td><td>Traing Accuracy</td><td>Testing Accuracy</td><td>NoSupport Vectors</td></tr>

<tr><td>'LINEAR'</td><td> 1</td><td> 0.5</td><td> 58.666666666666664</td><td> 62.8</td><td> 671</td></tr>
<tr><td>'LINEAR'</td><td> 10</td><td> 0.5</td><td> 59.73333333333334</td><td> 62.8</td><td> 650</td></tr>
<tr><td>'LINEAR'</td><td> 100</td><td> 0.5</td><td> 60.66666666666667</td><td> 64.0</td><td> 640</td></tr>
<tr><td>'LINEAR'</td><td> 100</td><td> 0.5</td><td> 60.66666666666667</td><td> 64.0</td><td> 640</td></tr>
<tr><td>'LINEAR'</td><td> 1000</td><td> 0.5</td><td> 61.73333333333333</td><td> 62.0</td><td> 637</td></tr>
<tr><td>'LINEAR'</td><td> 10000</td><td> 0.5</td><td> 61.46666666666667</td><td> 62.4</td><td> 635</td></tr>
<tr><td>'LINEAR'</td><td> 100000</td><td> 0.5</td><td> 56.266666666666666</td><td> 59.599999999999994</td><td> 608</td></tr>
<tr><td>'LINEAR'</td><td> 1000000</td><td> 0.5</td><td> 56.13333333333333</td><td> 57.599999999999994</td><td> 598</td></tr>
<tr><td>'LINEAR'</td><td> 0.1</td><td> 0.5</td><td> 57.733333333333334</td><td> 61.199999999999996</td><td> 724</td></tr>
<tr><td>'LINEAR'</td><td> 0.01</td><td> 0.5</td><td> 50.93333333333333</td><td> 48.4</td><td> 740</td></tr>
<tr><td>'LINEAR'</td><td> 0.001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'LINEAR'</td><td> 0.0001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'LINEAR'</td><td> 1e-05</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'LINEAR'</td><td> 1e-06</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'LINEAR'</td><td> 1e-07</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'LINEAR'</td><td> 1e-08</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>

<tr><td>'POLY'</td><td> 1</td><td> 0.5</td><td> 50.93333333333333</td><td> 48.4</td><td> 738</td></tr>
<tr><td>'POLY'</td><td> 10</td><td> 0.5</td><td> 51.33333333333333</td><td> 48.8</td><td> 736</td></tr>
<tr><td>'POLY'</td><td> 100</td><td> 0.5</td><td> 54.0</td><td> 54.800000000000004</td><td> 728</td></tr>
<tr><td>'POLY'</td><td> 1000</td><td> 0.5</td><td> 55.46666666666666</td><td> 55.2</td><td> 700</td></tr>
<tr><td>'POLY'</td><td> 10000</td><td> 0.5</td><td> 58.266666666666666</td><td> 56.39999999999999</td><td> 678</td></tr>
<tr><td>'POLY'</td><td> 100000</td><td> 0.5</td><td> 60.0</td><td> 56.39999999999999</td><td> 658</td></tr>
<tr><td>'POLY'</td><td> 1000000</td><td> 0.5</td><td> 65.33333333333333</td><td> 59.199999999999996</td><td> 625</td></tr>
<tr><td>'POLY'</td><td> 10000000</td><td> 0.5</td><td> 67.2</td><td> 56.8</td><td> 598</td></tr>
<tr><td>'POLY'</td><td> 100000000</td><td> 0.5</td><td> 72.13333333333334</td><td> 59.199999999999996</td><td> 579</td></tr>
<tr><td>'POLY'</td><td> 1000000000</td><td> 0.5</td><td> 74.53333333333333</td><td> 60.0</td><td> 541</td></tr>
<tr><td>'POLY'</td><td> 0.1</td><td> 0.5</td><td> 50.93333333333333</td><td> 48.4</td><td> 739</td></tr>
<tr><td>'POLY'</td><td> 0.01</td><td> 0.5</td><td> 50.8</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 0.001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 0.0001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 1e-05</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 1e-06</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 1e-07</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 1e-08</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 1e-09</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'POLY'</td><td> 1e-10</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>

<tr><td>'RBF'</td><td> 1</td><td> 0.5</td><td> 59.333333333333336</td><td> 59.199999999999996</td><td> 712</td></tr>
<tr><td>'RBF'</td><td> 10</td><td> 0.5</td><td> 59.73333333333334</td><td> 61.199999999999996</td><td> 668</td></tr>
<tr><td>'RBF'</td><td> 100</td><td> 0.5</td><td> 61.86666666666667</td><td> 59.599999999999994</td><td> 642</td></tr>
<tr><td>'RBF'</td><td> 1000</td><td> 0.5</td><td> 64.13333333333333</td><td> 60.4</td><td> 618</td></tr>
<tr><td>'RBF'</td><td> 10000</td><td> 0.5</td><td> 66.4</td><td> 64.0</td><td> 606</td></tr>
<tr><td>'RBF'</td><td> 100000</td><td> 0.5</td><td> 70.13333333333334</td><td> 61.199999999999996</td><td> 580</td></tr>
<tr><td>'RBF'</td><td> 1000000</td><td> 0.5</td><td> 72.0</td><td> 57.199999999999996</td><td> 543</td></tr>
<tr><td>'RBF'</td><td> 0.1</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 741</td></tr>
<tr><td>'RBF'</td><td> 0.01</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 741</td></tr>
<tr><td>'RBF'</td><td> 0.001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 741</td></tr>
<tr><td>'RBF'</td><td> 0.0001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'RBF'</td><td> 1e-05</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'RBF'</td><td> 1e-06</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'RBF'</td><td> 1e-07</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'RBF'</td><td> 1e-08</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>

<tr><td>'SIGMOID'</td><td> 1</td><td> 0.5</td><td> 57.599999999999994</td><td> 61.6</td><td> 722</td></tr>
<tr><td>'SIGMOID'</td><td> 10</td><td> 0.5</td><td> 57.733333333333334</td><td> 61.6</td><td> 668</td></tr>
<tr><td>'SIGMOID'</td><td> 100</td><td> 0.5</td><td> 59.73333333333334</td><td> 60.4</td><td> 515</td></tr>
<tr><td>'SIGMOID'</td><td> 1000</td><td> 0.5</td><td> 59.199999999999996</td><td> 60.8</td><td> 333</td></tr>
<tr><td>'SIGMOID'</td><td> 10000</td><td> 0.5</td><td> 59.46666666666667</td><td> 59.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 100000</td><td> 0.5</td><td> 59.333333333333336</td><td> 61.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 1000000</td><td> 0.5</td><td> 59.333333333333336</td><td> 61.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 10000000</td><td> 0.5</td><td> 59.333333333333336</td><td> 61.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 100000000</td><td> 0.5</td><td> 59.333333333333336</td><td> 61.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 1000000000</td><td> 0.5</td><td> 58.8</td><td> 61.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 10000000000</td><td> 0.5</td><td> 58.8</td><td> 61.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 100000000000</td><td> 0.5</td><td> 58.8</td><td> 61.199999999999996</td><td> 312</td></tr>
<tr><td>'SIGMOID'</td><td> 1000000000000</td><td> 0.5</td><td> 58.8</td><td> 61.199999999999996</td><td> 312</td></tr>

<tr><td>'SIGMOID'</td><td> 0.1</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 741</td></tr>
<tr><td>'SIGMOID'</td><td> 0.01</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 741</td></tr>
<tr><td>'SIGMOID'</td><td> 0.001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'SIGMOID'</td><td> 0.0001</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'SIGMOID'</td><td> 1e-05</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'SIGMOID'</td><td> 1e-06</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
<tr><td>'SIGMOID'</td><td> 1e-07</td><td> 0.5</td><td> 50.66666666666667</td><td> 48.0</td><td> 740</td></tr>
</table>

The following is the best of the above experiments (which are trained on random sampled data).

<table>

<tr><td>Kernel</td><td>C</td><td>Epsilon</td><td>Traing Accuracy</td><td>Testing Accuracy</td><td>NoSupport Vectors</td></tr>

<tr><td>'LINEAR'</td><td> 1000</td><td> 0.5</td><td> 61.73333333333333</td><td> 62.0</td><td> 637</td></tr>
<tr><td>'POLY'</td><td> 1000000000</td><td> 0.5</td><td> 74.53333333333333</td><td> 60.0</td><td> 541</td></tr>
<tr><td>'RBF'</td><td> 100000</td><td> 0.5</td><td> 70.13333333333334</td><td> 61.199999999999996</td><td> 580</td></tr>
<tr><td>'SIGMOID'</td><td> 10000</td><td> 0.5</td><td> 59.46666666666667</td><td> 59.199999999999996</td><td> 312</td></tr>

</table>